<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise_10_10_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

10- analogous to task 5 but predicting continuous subject age

In [1]:
%%capture
!pip install nilearn

In [ ]:
%%capture
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
import nibabel as nib

In [ ]:
%%capture
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import BayesianRidge
from datetime import datetime as dt

In [ ]:
%%capture
from IPython import display
display.set_matplotlib_formats('svg')

In [ ]:
# We import the data first in order to select the last 10 subjects
%%capture
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
crad = datasets.fetch_atlas_craddock_2012()
atlas_nii = index_img(crad['scorr_mean'], 1) # we fix this to 1
masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = StandardScaler().fit_transform(brain_data.ext_vars.age[:, None])[:, 0] # gives subject age on standard units after z-scoring

In [ ]:
# We prepare the X matrix and y vector, and we scale X
X, y = input_variables[:, 0:50], output_variable
X_scaled = scaler.fit_transform(X)

In [ ]:
# We create the train-test splits by hand
X_scaled_train = X_scaled[0:90,:]
X_scaled_test = X_scaled[90:100,:]
y_train = y[0:90]
y_test  = y[90:100]

In [ ]:
# We preallocate the variable where we will store the timings
running_secs = []

In [ ]:
# We are ready to iterate through the degree values:
for d in list(range(1,5)):

  X_scaled_train = X_scaled[0:90,:]                   # We re-instate X_scaled_train so that the initial value is always the same at the start of every iteration
  X_scaled_test = X_scaled[90:100,:]                # Ditto for X_scaled_test

  poly = PolynomialFeatures(degree=(d))                                 # We define the PolynomialFeatures model
  X_scaled_train = poly.fit_transform(X_scaled_train)         # We fit_transform the training data to a n-th degree polynomial 
  X_scaled_test = poly.fit_transform(X_scaled_test)             # Ditto for the test data

# We fit the logreg model on the transformed data to obtain training accuracies
  start = dt.now()      # We start the timer here
  b_ridge = BayesianRidge().fit(X_scaled_train, y_train)      # We fit the BayesianRidge model to the training data
  halt = ((dt.now() - start).seconds * 1000)          # We stop the timer here and make sure to multiply the seconds by 1000 to get values in miliseconds
  running_secs.append(halt)       # We append the current value of 'halt' to our pre-allocated 'running_secs' variable

In [ ]:
# We're ready to plot
sample=list(range(1,5))
plt.style.use("default")
plt.scatter(sample,running_secs, s=50, alpha = 0.5, marker = 'o')
plt.gca().set(
    title="Bayesian Ridge training model runtime (ms) as a function of polynomial degree (1-4) for 50 ROIs",
    xlabel="Polynomial degree (1, 2, 3 or 4)",
    ylabel="Miliseconds",
    xticks = (sample)
)
plt.plot(sample,running_secs)
plt.show()